In [1]:
########################################################################
# import python-library
########################################################################
import pickle
import os
import sys
import glob

import numpy as np
import librosa
import librosa.core
import librosa.feature
import yaml
import logging
import keras

from tqdm import tqdm
from sklearn import metrics
from keras.models import Model
from keras.layers import Input, Dense
########################################################################



########################################################################
# setup STD I/O
########################################################################
# 로깅을 설정하고 초기화하는 부분
logging.basicConfig(level=logging.DEBUG, filename="transfer_pretrain_v2.log")
logger = logging.getLogger(' ')
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
########################################################################




########################################################################
# file I/O
########################################################################
#파일 입출력 관련 함수 선언 부분
def save_pickle(filename, save_data):  
    logger.info("save_pickle -> {}".format(filename))
    with open(filename, 'wb') as sf:
        pickle.dump(save_data, sf)

def load_pickle(filename):
    logger.info("load_pickle <- {}".format(filename))
    with open(filename, 'rb') as lf:
        load_data = pickle.load(lf)
    return load_data

def file_load(wav_name, mono=False):
    try:
        return librosa.load(wav_name, sr=None, mono=mono)
    except:
        logger.error("file_broken or not exists!! : {}".format(wav_name))

def demux_wav(wav_name, channel=0):
    try:
        multi_channel_data, sr = file_load(wav_name)
        if multi_channel_data.ndim <= 1:
            return sr, multi_channel_data
        return sr, np.array(multi_channel_data)[channel, :]
    except ValueError as msg:
        logger.warning(f'{msg}')
########################################################################


 


########################################################################
# feature extractor
########################################################################
#소리 파일 하나를 로그멜스펙트로그램 형태로 바꾸고 딥러닝 모델에 넣을 형태로 바꾸는 함수 
def file_to_vector_array(file_name, n_mels=64, frames=5, n_fft=1024, hop_length=512, power=2.0):
    dims = n_mels * frames
    sr, y = demux_wav(file_name) 
    mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, power=power)
    log_mel_spectrogram = 20.0 / power * np.log10(mel_spectrogram + sys.float_info.epsilon)
    vectorarray_size = len(log_mel_spectrogram[0, :]) - frames + 1
    if vectorarray_size < 1:
        return np.empty((0, dims), float)
    vectorarray = np.zeros((vectorarray_size, dims), float)
    for t in range(frames):
        vectorarray[:, n_mels * t: n_mels * (t + 1)] = log_mel_spectrogram[:, t: t + vectorarray_size].T
    return vectorarray

#소리 파일들의 이름명이 담긴 리스트를 입력하면 그것들을 하나의 데이터셋으로 합치는 함수
def list_to_vector_array(file_list, msg="calc...", n_mels=64, frames=5, n_fft=1024, hop_length=512, power=2.0):
    dims = n_mels * frames
    for idx in tqdm(range(len(file_list)), desc=msg):
        vector_array = file_to_vector_array(file_list[idx], n_mels=n_mels, frames=frames, n_fft=n_fft, hop_length=hop_length, power=power)
        if idx == 0:
            dataset = np.zeros((vector_array.shape[0] * len(file_list), dims), float)          
        dataset[vector_array.shape[0] * idx: vector_array.shape[0] * (idx + 1), :] = vector_array
    return dataset


# machine4종류와 id4종류 각각의 데이터셋을 만드는 함수. 
# eval데이터셋: abnormal파일들 + normal파일들(5대5비율), train데이터셋: eval에 사용한 normal파일을 제외한 나머지 normal파일 전부
def dataset_generator(target_dir, machine_type, id_, normal_dir_name="normal", abnormal_dir_name="abnormal", ext="wav"):
    logger.info("target_dir : {}".format(target_dir))
    machine_id = f"-6dB_{machine_type}"
    machine = f"{machine_type}"
    id_ = f"{id_}"
    machine_type_id_path = os.path.join(target_dir, machine_id, machine, id_)
    
    normal_files = sorted(glob.glob(os.path.abspath("{dir}/{normal_dir_name}/*.{ext}".format(dir=machine_type_id_path, normal_dir_name=normal_dir_name, ext=ext))))
    normal_labels = np.zeros(len(normal_files))
    if len(normal_files) == 0:
        logger.exception("no_wav_data!!")
        
    abnormal_files = sorted(glob.glob(os.path.abspath("{dir}/{abnormal_dir_name}/*.{ext}".format(dir=machine_type_id_path, abnormal_dir_name=abnormal_dir_name, ext=ext))))
    abnormal_labels = np.ones(len(abnormal_files))
    if len(abnormal_files) == 0:
        logger.exception("no_wav_data!!")
        
    train_files = normal_files[len(abnormal_files):]
    train_labels = normal_labels[len(abnormal_files):]
    eval_files = np.concatenate((normal_files[:len(abnormal_files)], abnormal_files), axis=0)
    eval_labels = np.concatenate((normal_labels[:len(abnormal_files)], abnormal_labels), axis=0)
    
    logger.info("train_file num : {num}".format(num=len(train_files)))
    logger.info("eval_file  num : {num}".format(num=len(eval_files)))

    return train_files, train_labels, eval_files, eval_labels
########################################################################



########################################################################
# keras model
########################################################################
#오토인코더 함수
def keras_model(inputDim):
    inputLayer = Input(shape=(inputDim,))
    h = Dense(64, activation="relu")(inputLayer)    
    h = Dense(64, activation="relu")(h)
    h = Dense(8, activation="relu")(h)
    h = Dense(64, activation="relu")(h)
    h = Dense(64, activation="relu")(h)
    h = Dense(inputDim, activation=None)(h)
    return Model(inputs=inputLayer, outputs=h)
########################################################################




########################################################################
# main
########################################################################
# 메인 실행 부분
if __name__ == "__main__":
    with open("make_pretrain_v2.yaml", encoding='utf-8') as stream:
        pretrain_v2_param = yaml.safe_load(stream)
    with open("transfer_pretrain_v2.yaml", encoding='utf-8') as stream:
        transfer_v2_param = yaml.safe_load(stream)

    # pickle데이터, model데이터, result데이터가 저장될 파일과 폴더들 관련 변수 선언
    os.makedirs(transfer_v2_param["pickle_directory"], exist_ok=True)
    os.makedirs(transfer_v2_param["model_directory"], exist_ok=True)
    os.makedirs(transfer_v2_param["result_directory"], exist_ok=True)
    model_directory = transfer_v2_param["model_directory"]
    base_directory = transfer_v2_param["base_directory"]
    result_file = "{result}/{file_name}".format(result=transfer_v2_param["result_directory"], file_name=transfer_v2_param["result_file"])
    results = {}
    machine_types = ["fan", "valve", "slider", "pump"]    
    id_list =  ["id_00", "id_02", "id_04", "id_06"]   

    
    # 4종류의 machine와 4종류의 id를 각각 for문을 돔.
    for machine_type in machine_types:
        for id_ in id_list:
            print("\n===========================")
            train_pickle = "{pickle}/train_{machine_type}_{id_}.pickle".format(pickle=transfer_v2_param["pickle_directory"], machine_type= machine_type, id_ = id_)
            train_labels_pickle = "{pickle}/train_labels_{machine_type}_{id_}.pickle".format(pickle=transfer_v2_param["pickle_directory"], machine_type= machine_type, id_ = id_)
            eval_files_pickle = "{pickle}/eval_files_{machine_type}_{id_}.pickle".format(pickle=transfer_v2_param["pickle_directory"], machine_type= machine_type, id_ = id_)
            eval_labels_pickle = "{pickle}/eval_labels_{machine_type}_{id_}.pickle".format(pickle=transfer_v2_param["pickle_directory"], machine_type= machine_type, id_ = id_)
            
            #데이터셋 생성, pickle에 저장돼있다면 그것을 사용
            if os.path.exists(train_pickle) and os.path.exists(eval_files_pickle) and os.path.exists(eval_labels_pickle):
                train_data = load_pickle(train_pickle)
                eval_files = load_pickle(eval_files_pickle)
                eval_labels = load_pickle(eval_labels_pickle)
                train_labels = load_pickle(train_labels_pickle)
            else:
                train_files, train_labels, eval_files, eval_labels = dataset_generator(base_directory, machine_type, id_)
                train_data = list_to_vector_array(train_files,
                                                  msg="generate train_dataset",
                                                  n_mels=transfer_v2_param["feature"]["n_mels"],
                                                  frames=transfer_v2_param["feature"]["frames"],
                                                  n_fft=transfer_v2_param["feature"]["n_fft"],
                                                  hop_length=transfer_v2_param["feature"]["hop_length"],
                                                  power=transfer_v2_param["feature"]["power"])
                save_pickle(train_pickle, train_data)
                save_pickle(eval_files_pickle, eval_files)
                save_pickle(eval_labels_pickle, eval_labels)
                save_pickle(train_labels_pickle, train_labels)

            # 만약 machine이 fan이고 id가00이라면 fan_00을 제외한 fan_02~06으로 학습한 pretrain모델을 불러와서 학습시킴. 같은 기계타입, 다른 아이디로 학습된 모델
            model = keras.models.load_model(pretrain_v2_param["model_directory"] + "/pretrain_except_{machine_type}_{id_}.h5".format(machine_type= machine_type, id_ = id_))
            model_file = "{model}/transfer_{machine_type}_{machine_id}_to_general_5.h5".format(model=model_directory,
                                                                                      machine_type=machine_type,
                                                                                      machine_id=id_)
            # if not os.path.exists(model_file):
            model.compile(**transfer_v2_param["fit"]["compile"])
            model.fit(train_data, train_data, epochs=transfer_v2_param["fit"]["epochs"], batch_size=transfer_v2_param["fit"]["batch_size"], shuffle=transfer_v2_param["fit"]["shuffle"],validation_split=transfer_v2_param["fit"]["validation_split"],verbose=transfer_v2_param["fit"]["verbose"])
            model.save(model_file)

            print("============== EVALUATION ==============")
            y_pred = [0. for k in eval_labels]
            y_true = eval_labels

            evaluation_result_key = "{machine_type}_{id_}_to_general_{machine_type}".format(machine_type= machine_type, id_= id_)
            evaluation_result = {}
            for num, file_name in tqdm(enumerate(eval_files), total=len(eval_files)):
                try:
                    data = file_to_vector_array(file_name,
                                                n_mels=transfer_v2_param["feature"]["n_mels"],
                                                frames=transfer_v2_param["feature"]["frames"],
                                                n_fft=transfer_v2_param["feature"]["n_fft"],
                                                hop_length=transfer_v2_param["feature"]["hop_length"],
                                                power=transfer_v2_param["feature"]["power"])
                    error = np.mean(np.square(data - model.predict(data)), axis=1)
                    y_pred[num] = np.mean(error)
                except:
                    logger.warning("File broken!!: {}".format(file_name))

            # auc값을 저장함
            score = metrics.roc_auc_score(y_true, y_pred)
            logger.info("AUC : {}".format(score))
            evaluation_result["AUC"] = float(score)
            results[evaluation_result_key] = evaluation_result
            print("===========================")
        
        # 모든 학습이 완료됐을 때 auc 결과들을 yaml파일로 저장
        logger.info("all results -> {}".format(result_file))
        with open(result_file, "w") as f:
            f.write(yaml.dump(results, default_flow_style=False))       
        print("===========================")
########################################################################

2023-08-26 00:59:15,677 - INFO - target_dir : ../Sample_data
2023-08-26 00:59:15,681 - INFO - train_file num : 5
2023-08-26 00:59:15,683 - INFO - eval_file  num : 10


generate train_dataset: 100%|████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.89it/s]
2023-08-26 00:59:18,338 - INFO - save_pickle -> ./pickle_transfer_pretrain_v2/train_fan_id_00.pickle
2023-08-26 00:59:18,343 - INFO - save_pickle -> ./pickle_transfer_pretrain_v2/eval_files_fan_id_00.pickle
2023-08-26 00:59:18,345 - INFO - save_pickle -> ./pickle_transfer_pretrain_v2/eval_labels_fan_id_00.pickle
2023-08-26 00:59:18,348 - INFO - save_pickle -> ./pickle_transfer_pretrain_v2/train_labels_fan_id_00.pickle


Epoch 1/50
3/3 [==============================] - 1s 77ms/step - loss: 17.2342 - val_loss: 20.3495
Epoch 2/50
3/3 [==============================] - 0s 15ms/step - loss: 16.0614 - val_loss: 20.7839
Epoch 3/50
3/3 [==============================] - 0s 15ms/step - loss: 15.2711 - val_loss: 17.8787
Epoch 4/50
3/3 [==============================] - 0s 15ms/step - loss: 13.9259 - val_loss: 17.5569
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: 13.3429 - val_loss: 18.7694
Epoch 6/50
3/3 [==============================] - 0s 16ms/step - loss: 13.4603 - val_loss: 16.9353
Epoch 7/50
3/3 [==============================] - 0s 15ms/step - loss: 12.7849 - val_loss: 16.4325
Epoch 8/50
3/3 [==============================] - 0s 15ms/step - loss: 12.7483 - val_loss: 17.6436
Epoch 9/50
3/3 [==============================] - 0s 15ms/step - loss: 12.5313 - val_loss: 16.8287
Epoch 10/50
3/3 [==============================] - 0s 16ms/step - loss: 12.3250 - val_loss: 15.8546
Epoch 11/

C:\Users\Baejs\anaconda3\envs\bae\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

10/10 [==============================] - 0s 1ms/step


...

 70%|██████████████████████████████████████████████████████████                         | 7/10 [00:00<00:00, 12.44it/s]

10/10 [==============================] - 0s 997us/step


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:00<00:00, 12.59it/s]

10/10 [==============================] - 0s 1ms/step


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.00it/s]
2023-08-26 01:00:20,106 - INFO - AUC : 0.44
2023-08-26 01:00:20,106 - INFO - all results -> ./result_transfer_pretrain_v2/result_transfer_pretrain_v2.yaml
